In [56]:
#importing all package....
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os
# Import Keras and other Deep Learning dependencies
from keras.models import Sequential
import time
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
import seaborn as sns
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from sklearn.preprocessing import LabelBinarizer
from keras.optimizers import *
from keras.engine.topology import Layer
from keras import backend as K
from keras.regularizers import l2
K.set_image_data_format('channels_last')
import cv2
import os
from skimage import io
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf


import numpy.random as rng
from sklearn.utils import shuffle
from keras.layers import *
from keras.models import Model
import cv2
from PIL import Image

In [57]:
#preparing train data...................

train_data=pd.read_csv('train_relationships.csv')
test_data=pd.read_csv('sample_submission.csv')

train_data[:5]

,p1,p2
0,F0002/MID1,F0002/MID3
1,F0002/MID2,F0002/MID3
2,F0005/MID1,F0005/MID2
3,F0005/MID3,F0005/MID2
4,F0009/MID1,F0009/MID4


In [58]:
train_data.describe()
train_data.isnull().sum()



p1    0
p2    0
dtype: int64

In [59]:
#add non-related column.......
list1=list(train_data.p1)

list2=list(train_data.p2)
new_list=list1[::-1]


temp_data=pd.DataFrame({'p1':new_list,'p2':list2})
#print(temp_data)
train_data=pd.concat([train_data,temp_data])
train_data.describe()

#creating whole label................................................
label=[]

for i in range(0,7196):
    if i<3599:
        label.append(1)
    else:
        label.append(0)
#print(label)

In [60]:
def add_image_path(x):
    image_path='train/'+x
    temp_path='train/F0002/MID1/P00017_face3.jpg'
    #print(image_path)
    if os.path.exists(image_path):
        #print(os.listdir(image_path)[0])
        path=os.path.join(image_path,os.listdir(image_path)[0])
        #print(path)
        return path
    else:
        return temp_path

In [61]:
train_data['p1_path']=train_data.p1.apply(lambda x:add_image_path(x))

train_data['p2_path']=train_data.p2.apply(lambda x:add_image_path(x))


train_data['is_related']=np.array(label)


In [62]:
#now check the train_data 
train_data.head()

,p1,p2,p1_path,p2_path,is_related
0,F0002/MID1,F0002/MID3,train/F0002/MID1/P00017_face3.jpg,train/F0002/MID3/P00013_face3.jpg,1
1,F0002/MID2,F0002/MID3,train/F0002/MID2/P00018_face3.jpg,train/F0002/MID3/P00013_face3.jpg,1
2,F0005/MID1,F0005/MID2,train/F0005/MID1/P00059_face2.jpg,train/F0005/MID2/P00054_face2.jpg,1
3,F0005/MID3,F0005/MID2,train/F0005/MID3/P00059_face1.jpg,train/F0005/MID2/P00054_face2.jpg,1
4,F0009/MID1,F0009/MID4,train/F0009/MID1/P11724_face4.jpg,train/F0009/MID4/P11744_face1.jpg,1


In [63]:
train_data.fillna(method='ffill', axis=1)
train_data.isnull().sum()


p1            0
p2            0
p1_path       0
p2_path       0
is_related    0
dtype: int64

In [64]:
train_data.tail(10)
train_data.isnull().sum()

p1            0
p2            0
p1_path       0
p2_path       0
is_related    0
dtype: int64

In [66]:
#making x_train and y_train..here..........................

train_image_1=[]
train_image_2=[]
train_label=[]

h=0
w=0

for image_1,image_2,label in zip(train_data['p1_path'],train_data['p2_path'],train_data['is_related']):
    
        img_1=Image.open(image_1)
        #print(image_1,image_2)
        img_2=Image.open(image_2)
        new1=np.array(img_1,dtype='uint8')
        new2=np.array(img_2,dtype='uint8')
        
        width, height = img_1.size
        if width>w:
            w=width
        if height>h:
            h=height
        
        train_image_1.append(new1)
        train_image_2.append(new2)
        
        train_label.append(label)


        
        
print(h,w)
train_image_1= np.array(train_image_1)
train_image_2= np.array(train_image_2)



224 224


# Completed of making training dataset

In [79]:

#model-1 structure is defining here.........................

model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3))) 
model1.add(MaxPooling2D((2, 2), padding='same'))
model1.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2), padding='same'))

#model-2 structure is defining here..................
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3))) 
model2.add(MaxPooling2D((2, 2), padding='same'))
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), padding='same'))

#merging the two model................................
mergeOut=Add()([model1.output,model2.output])

mergeout=Flatten()(mergeOut)
mergeout=Dense(256,activation='relu')(mergeout)
mergeout=Dropout(.5)(mergeOut)
mergeout=Dense(128,activation='relu')(mergeout)
mergeout=Dropout(.35)(mergeOut)

#output layer.......
mergeout=Dense(2,activation='softmax')(mergeOut)

newModel=Model([model1.input,model2.input],mergeOut)

newModel.summary() 

newModel.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=["accuracy"])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_17_input (InputLayer)    (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_19_input (InputLayer)    (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 224, 224, 32) 896         conv2d_17_input[0][0]            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 224, 224, 32) 896         conv2d_19_input[0][0]            
__________________________________________________________________________________________________
max_poolin

In [80]:
newModel.fit([train_image_1,train_image_2],train_label,epochs=10, batch_size=52, shuffle=True)

ValueError: Error when checking target: expected add_5 to have 4 dimensions, but got array with shape (7196, 1)